In [3]:
import socket
import ssl
from cryptography.x509 import load_pem_x509_certificate
%run FunçõesAuxiliares.ipynb 
import os
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.x509.oid import NameOID

In [4]:
pasta_csrs = "csrs"
pasta_certs = "certificados"
pasta_ca = "AutoridadeCertificadora"
os.makedirs(pasta_csrs, exist_ok=True)
os.makedirs(pasta_certs, exist_ok=True)
os.makedirs(pasta_ca, exist_ok=True)

def gerar_chave_e_csr(common_name, country_name, state_or_province_name, locality_name, organization_name):
    key_filename = os.path.join(pasta_certs, f"{common_name}_chave.pem")   # chave salva aqui
    csr_filename = os.path.join(pasta_csrs, f"{common_name}.csr")    # CSR salva aqui

    # Se já existe chave e csr, não gera de novo
    if os.path.exists(key_filename) and os.path.exists(csr_filename):
        print(f"Chave e CSR para {common_name} já existem.")
        return key_filename, csr_filename

    print("Gerando chave privada...")
    key = rsa.generate_private_key(
        public_exponent=65537,
        key_size=4096
    )

    print("Criando CSR...")
    subject = x509.Name([
        x509.NameAttribute(NameOID.COUNTRY_NAME, country_name),
        x509.NameAttribute(NameOID.STATE_OR_PROVINCE_NAME, state_or_province_name),
        x509.NameAttribute(NameOID.LOCALITY_NAME, locality_name),
        x509.NameAttribute(NameOID.ORGANIZATION_NAME, organization_name),
        x509.NameAttribute(NameOID.COMMON_NAME, common_name)
    ])

    csr = x509.CertificateSigningRequestBuilder().subject_name(subject).sign(key, hashes.SHA256(), default_backend())

    with open(key_filename, "wb") as f:
        f.write(key.private_bytes(
            encoding=serialization.Encoding.PEM,
            format=serialization.PrivateFormat.TraditionalOpenSSL,
            encryption_algorithm=serialization.NoEncryption()
        ))

    with open(csr_filename, "wb") as f:
        f.write(csr.public_bytes(serialization.Encoding.PEM))

    print(f"Chave e CSR criados: {key_filename}, {csr_filename}")
    print(f"Envie o CSR (arquivo '{csr_filename}') para a CA para assinar o certificado.")

    return key_filename, csr_filename


def esperar_certificado(common_name):
    cert_filename = os.path.join(pasta_certs, f"{common_name}_cert.pem")
    print(f"\nAguardando certificado assinado pela CA... ({cert_filename})")

    while not os.path.exists(cert_filename):
        print(f"Certificado ainda não encontrado. Por favor, copie o certificado emitido para este caminho: {cert_filename}")
        resposta = input("Pressione Enter para verificar novamente ou digite 'sair' para cancelar: ").strip().lower()
        if resposta == 'sair':
            print("Cancelando espera pelo certificado.")
            return None  # Retorna None para indicar cancelamento
    print(f"Certificado encontrado: {cert_filename}")
    return cert_filename

def chat_continuo_cliente(certfile, keyfile, cafile, common_name_dest, caminho_cert_dest, chave_aes_servidor, chave_pub_dest):
    context = ssl.create_default_context()
    context.load_cert_chain(certfile=certfile, keyfile=keyfile)
    context.load_verify_locations(cafile=cafile)
    context.verify_mode = ssl.CERT_REQUIRED

    if not(validacoes(caminho_cert_dest,cafile, common_name_dest)):
        print('O Certificado Digital de quem deseja comunicar não passou as devidas validações. Conexão não pode ser iniciada.')
        return

    sock = socket.create_connection(('192.168.1.199', 4443))
    conn = context.wrap_socket(sock, server_hostname= common_name_dest)  

    try:
        while True:
            texto = input("Você: ")
            if texto.lower() == "sair":
                break

            mensagem_criptografada = enviar_pacote_json(chave_aes_servidor, texto, chave_pub_dest)
            print('\n',mensagem_criptografada,'\n')
            conn.sendall(mensagem_criptografada)

            resposta_criptografada = conn.recv(4096)
            resposta = desencriptar_mensagem(resposta_criptografada.decode(), keyfile)
            print(f"Servidor diz: {resposta}")

    finally:
        conn.close() 


if __name__ == "__main__":
    print("=== Cliente TLS com CSR e chat seguro ===")

    common_name = input("Common Name (CN): ")
    common_name_dest = input("Common Name (CN) do Destinatário: ")

    caminho_cert_cliente = os.path.join(pasta_certs, f"{common_name}_cert.pem")
    caminho_key_cliente = os.path.join(pasta_certs, f"{common_name}_chave.pem")
    caminho_csr_cliente = os.path.join(pasta_csrs, f"{common_name}.csr")
    chave_aes_servidor = gerar_chave_aes()
    caminho_cert_ca = "AutoridadeCertificadora/certificado_raiz.pem"
    caminho_cert_dest = os.path.join(pasta_certs, f"{common_name_dest}_cert.pem")

    if not os.path.exists(caminho_cert_cliente):
        # Se não tem certificado, então gera chave e CSR e espera pelo certificado
        country_name = input("Country Name (2-letter code): ")
        state_or_province_name = input("State or Province Name: ")
        locality_name = input("Locality Name: ")
        organization_name = input("Organization Name: ")

        caminho_key_cliente, caminho_csr_cliente = gerar_chave_e_csr(
            common_name, country_name, state_or_province_name, locality_name, organization_name
        )
        caminho_cert_cliente = esperar_certificado(common_name)
        if caminho_cert_cliente is None:
            print("Certificado não foi encontrado. Encerrando.")
            exit(0)
    else:
        print(f"Certificado já existe: {caminho_cert_cliente}")

    if not os.path.exists(caminho_cert_ca):
        print(f"Erro: Certificado da CA raiz não encontrado em {caminho_cert_ca}.")
        exit(1)

    if not os.path.exists(caminho_cert_dest):
        print(f"Erro: Certificado do destinatário não encontrado em {caminho_cert_dest}.")
        exit(1)

    with open(caminho_cert_dest, "rb") as f:
        cert_dest = load_pem_x509_certificate(f.read())
        chave_pub_dest = cert_dest.public_key()

    chat_continuo_cliente(
        certfile=caminho_cert_cliente,
        keyfile=caminho_key_cliente,
        cafile=caminho_cert_ca,
        common_name_dest=common_name_dest,
        caminho_cert_dest=caminho_cert_dest,
        chave_aes_servidor=chave_aes_servidor,
        chave_pub_dest=chave_pub_dest
    )

=== Cliente TLS com CSR e chat seguro ===


Common Name (CN):  Lourenco
Common Name (CN) do Destinatário:  Marianna


Certificado já existe: certificados\Lourenco_cert.pem


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [ ]:
#ficheiro ficheiros_teste/teste.txt